<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sodapy

In [0]:
import time as time
import numpy as np
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [0]:
def get_random_samples(client, num_samples=1000, sample_size=1000):
    
    start = time.time()

    # Perform a $select=count(*) query to determine how large the set is
    results = client.get("m6dm-c72p", select='count(*)' )
    total_rows = int(results[0].get('count', 0))
    row_indices = np.arange(0, total_rows, sample_size)
    results = []


    # Use rand() locally to come up with some offsets
    sample_offsets = np.random.choice(row_indices, size=num_samples, replace=False)


    # Use $limit and $offset in conjunction with a stable $order to pick out individual records. 
    # Ex: $order=facility_id&$limit=1&$offset=<some rand() number>
    for i, offset in enumerate (sample_offsets):

        print(f'Sample {i}: offset={offset},sample_size={sample_size}')
        print('Pure Python time:', time.time() - start, 'sec.')
        results.extend(client.get("m6dm-c72p", order='trip_id', limit=sample_size, offset=offset, 
                                                  select='''trip_id, trip_start_timestamp, pickup_community_area, fare, tip, trip_total'''))
        
    
    # results.extend(client.get("m6dm-c72p", order='trip_id', limit=sample_size, offset=sample_offsets[5]))

    print('Pure Python time:', time.time() - start, 'sec.')

    return results

In [0]:
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")
client.timeout = 1000

In [0]:
samples = get_random_samples(client)
samples[:10]

Sample 0: offset=1138000,sample_size=1000
Pure Python time: 0.35587525367736816 sec.
Sample 1: offset=23603000,sample_size=1000
Pure Python time: 1.0074210166931152 sec.
Sample 2: offset=84632000,sample_size=1000
Pure Python time: 11.57935380935669 sec.
Sample 3: offset=9182000,sample_size=1000
Pure Python time: 50.080636978149414 sec.
Sample 4: offset=89828000,sample_size=1000
Pure Python time: 54.03202223777771 sec.
Sample 5: offset=66460000,sample_size=1000
Pure Python time: 101.94308996200562 sec.
Sample 6: offset=94242000,sample_size=1000
Pure Python time: 130.5291771888733 sec.
Sample 7: offset=74765000,sample_size=1000
Pure Python time: 174.4577043056488 sec.
Sample 8: offset=31295000,sample_size=1000
Pure Python time: 206.36658024787903 sec.
Sample 9: offset=1131000,sample_size=1000
Pure Python time: 220.1940577030182 sec.
Sample 10: offset=65973000,sample_size=1000
Pure Python time: 220.87951636314392 sec.
Sample 11: offset=85288000,sample_size=1000
Pure Python time: 247.58682

In [100]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=2000000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16T16:15:00.000,2019-01-16T16:30:00.000,1346,8.21715513808,7,77,15,0,2.55,17.55
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01T06:00:00.000,2018-11-01T07:00:00.000,3113,20.4166496394989,45,23,17.5,0,2.5,20
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24T19:45:00.000,2019-01-24T19:45:00.000,469,1.5670652572791,31,31,5,0,2.55,7.55
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21T18:15:00.000,2018-12-21T18:30:00.000,880,4.1697883216,69,39,10,0,0,10
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05T14:45:00.000,2019-03-05T15:00:00.000,1506,7.32213988795855,28,29,10,0,2.55,12.55
...,...,...,...,...,...,...,...,...,...,...,...
1999995,1143f37b3053e2f9d4f37f613cdd5825c26dda78,2019-08-12T01:45:00.000,2019-08-12T02:00:00.000,1015,6.722369,5,8,12.5,3,2.55,18.05
1999996,1143f38297b979a3b88696b2244bd02341d2a500,2019-07-19T09:15:00.000,2019-07-19T09:30:00.000,1306,7.388153,32,6,15,5,2.55,22.55
1999997,1143f48e90eb89cccbff886fc4147e3c5a27fa35,2019-09-03T16:15:00.000,2019-09-03T17:00:00.000,2786,14.704924,77,76,27.5,0,7.55,35.05
1999998,1143f5069330b99ec01ae6e2da8e24836719f852,2019-09-29T18:15:00.000,2019-09-29T18:45:00.000,1753,23.44713608976,76,NaN,27.5,0,9.14,36.64


In [111]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip', 'apparentTemperature']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precip,apparentTemperature
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16 16:15:00,2019-01-16 16:30:00,1346,8.217155,7,15.0,0.0,2.55,17.55,2,16,5,2019-01-16 16:00:00,clear,21.94
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01 06:00:00,2018-11-01 07:00:00,3113,20.416650,45,17.5,0.0,2.50,20.00,3,6,2,2018-11-01 06:00:00,clear,50.02
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24 19:45:00,2019-01-24 19:45:00,469,1.567065,31,5.0,0.0,2.55,7.55,3,19,6,2019-01-24 19:00:00,clear,-1.47
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21 18:15:00,2018-12-21 18:30:00,880,4.169788,69,10.0,0.0,0.00,10.00,4,18,6,2018-12-21 18:00:00,clear,21.29
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05 14:45:00,2019-03-05 15:00:00,1506,7.322140,28,10.0,0.0,2.55,12.55,1,14,4,2019-03-05 14:00:00,clear,6.71


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 0 to 1999999
Data columns (total 16 columns):
trip_id                  object
trip_start_timestamp     datetime64[ns]
trip_end_timestamp       datetime64[ns]
trip_seconds             int64
trip_miles               float64
pickup_community_area    object
fare                     float64
tip                      float64
additional_charges       float64
trip_total               float64
start_weekday            int64
start_hour               int64
start_time_block         int64
start_date_plus_hour     datetime64[ns]
precip                   object
apparentTemperature      float64
dtypes: datetime64[ns](3), float64(6), int64(4), object(3)
memory usage: 259.4+ MB


In [214]:
columns_to_use = ['apparentTemperature', 'start_weekday', 'start_time_block', 'pickup_community_area']
columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X = df.drop(columns=columns_to_drop)
X['apparentTemperature']=X['apparentTemperature'].fillna(X['apparentTemperature'].median())
X['pickup_community_area']=X['pickup_community_area'].fillna('0')

cont_cols = ['apparentTemperature']
cat_cols = [ col for col in columns_to_use if col not in cont_cols ]

enc = OneHotEncoder()

y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train_cont = X_train[cont_cols]
X_train_cat = X_train[cat_cols]
X_test_cont = X_test[cont_cols]
X_test_cat = X_test[cat_cols]


# Transform training and test sets
X_train_enc = enc.fit_transform(X_train_cat, y_train)
X_test_enc = enc.transform(X_test_cat)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test_cat = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

# Combine categorical and continuous features into the final dataframe
X_train = pd.concat([X_train_cont, X_train_cat], axis=1)
X_test = pd.concat([X_test_cont, X_test_cat], axis=1)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

model = sm.OLS(y_train,X_train).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.277
Method:                 Least Squares   F-statistic:                     6328.
Date:                Wed, 22 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:31:58   Log-Likelihood:            -5.5612e+06
No. Observations:             1500000   AIC:                         1.112e+07
Df Residuals:                 1499908   BIC:                         1.112e+07
Df Model:                          91                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       10.6224      0.032    329.300      0.000      10.559      10.686
apparentTemperature          0.0168      0.001     32.640      0.000       0.016       0.018
start_weekday_0              1.6677      0.022     77.075      0.000       1.625       1.710
start_weekday_1              1.1430      0.022     52.149      0.000       1.100       1.186
start_weekday_2              1.3957      0.021     65.055      0.000       1.354       1.438
start_weekday_3              1.7881      0.020     87.385      0.000       1.748       1.828
start_weekday_4              1.6466      0.019     85.663      0.000       1.609       1.684
start_weekday_5              1.3348      0.019     71.048      0.000       1.298       1.372
start_weekday_6              1.6465      0.020     81.802      0.000       1.607       1.686
start_time_block_0           0.0704      0.027      2.576      0.010       0.017       0.124
start_time_block_1           4.4388      0.036    121.850      0.000       4.367       4.510
start_time_block_2           2.2161      0.023     95.079      0.000       2.170       2.262
start_time_block_3           1.1924      0.022     55.254      0.000       1.150       1.235
start_time_block_4           1.4105      0.022     65.270      0.000       1.368       1.453
start_time_block_5           1.4721      0.020     72.559      0.000       1.432       1.512
start_time_block_6          -0.0592      0.019     -3.069      0.002      -0.097      -0.021
start_time_block_7          -0.1186      0.020     -5.850      0.000      -0.158      -0.079
pickup_community_area_0     13.4951      0.037    367.416      0.000      13.423      13.567
pickup_community_area_1      0.1264      0.081      1.570      0.116      -0.031       0.284
pickup_community_area_10     0.9901      0.146      6.767      0.000       0.703       1.277
pickup_community_area_11     0.1232      0.154      0.801      0.423      -0.178       0.425
pickup_community_area_12     2.4011      0.218     11.034      0.000       1.975       2.828
pickup_community_area_13    -0.4006      0.175     -2.292      0.022      -0.743      -0.058
pickup_community_area_14    -0.3993      0.106     -3.779      0.000      -0.606      -0.192
pickup_community_area_15     0.0588      0.104      0.564      0.573      -0.146       0.263
pickup_community_area_16    -0.3336      0.085     -3.947      0.000      -0.499      -0.168
pickup_community_area_17     0.0643      0.157      0.408      0.683      -0.244       0.373
pickup_community_area_18    -0.3918      0.249     -1.571      0.116      -0.881       0.097
pickup_community_area_19    -1.4408      0.097    -14.848      0.000      -1.631      -1.251
pickup_community_area_2     -0.8735      0.094     -9.296      0.000      -1.058      -0.689
pickup_communi

In [217]:
# Fit the model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train, y_train))
print('Test r^2:', linreg.score(X_test, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test)))


Training r^2: 0.2774238170015998
Test r^2: 0.274333459655718
Training MSE: 97.22444938935793
Test MSE: 97.70969693903413


In [0]:
# Fit the model
# linreg = LinearRegression()

# X_train_cont = sm.add_constant(X_train_cont)
# X_test_cont = sm.add_constant(X_test_cont)

# linreg.fit(X_train_cont, y_train)

# # Print R2 and MSE for training and test sets
# print('Training r^2:', linreg.score(X_train_cont, y_train))
# print('Test r^2:', linreg.score(X_test_cont, y_test))
# print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_cont)))
# print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_cont)))
